In [6]:
#-*- coding: utf-8 -*-
import requests
import os, sys, re
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px

from glob import glob
from tqdm.auto import tqdm

from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import ColumnDataSource, HoverTool, BoxAnnotation
from bokeh.palettes import Category10
from bokeh.layouts import layout
from bokeh.io.export import export_png

sys.path.append(os.path.abspath("../../code"))

import functions as fc
import naverApi_function as nv

# pandas 설정
pd.set_option('display.max_columns', 100)

# matplotlib 설정
plt.rcParams['font.family'] = 'NanumGothic'  # 나눔고딕 설정
plt.rcParams['axes.unicode_minus'] = False   # 마이너스 기호 깨짐 방지

In [2]:
data_path = os.path.abspath("../../data")

In [8]:
glob(f"{data_path}/raw/10000개의레시피/*")

['/Users/stevenkim/Documents/dev/company/code/private/innerCircle/data/raw/10000개의레시피/TB_RECIPE_SEARCH-220701.csv',
 '/Users/stevenkim/Documents/dev/company/code/private/innerCircle/data/raw/10000개의레시피/TB_RECIPE_SEARCH_241226.csv',
 '/Users/stevenkim/Documents/dev/company/code/private/innerCircle/data/raw/10000개의레시피/TB_RECIPE_SEARCH-20231130.csv']

In [13]:
file = f"{data_path}/raw/10000개의레시피/TB_RECIPE_SEARCH_241226.csv"
encoding_detected = fc.detect_encoding(file)
df = pd.read_csv(file, encoding=encoding_detected)
df.columns = df.columns.str.lower()
df

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,20240101000857,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,7016814,#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육,된장수육,kstencil,강철새잎,1396,0,1,삶기,술안주,돼지고기,메인반찬,수육용 삼겹살을 사다가 된장과 술을 넣고 일상먹거리 수육한접시를 만들어 주었습니다....,[재료] 돼지고기 수육용삼겹살500g| 된장1.5큰술| 술4큰술| ...,2인분,아무나,2시간이내,20240101002917,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...
3,7016816,만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품,만두전골,aeaelove,춤추는루나,6350,0,6,끓이기,손님접대,가공식품류,국/탕,속이 꽉 찬 만두가 듬뿍 푸짐한 백종원 만두전골 만두전골 레시피 만두 요리는 다양한...,[재료] 만두12개| 청경채3개| 양파1/2개| 대파1/2개|...,4인분,초급,60분이내,20240101073609,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,7016817,새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기,무수분보쌈,aeaelove,춤추는루나,1829,0,6,삶기,손님접대,돼지고기,메인반찬,촉촉하고 부드러운 통삼겹살 무수분 보쌈 삶는법 무수분 보쌈 삶는 시간 백종원 보쌈 ...,[재료] 통삼겹살600g| 양파1개| 못난이 사과 小2개| 대파2...,4인분,초급,90분이내,20240101074507,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23187,7041366,도토리묵무침 도토리묵 양념장 만들기 도토리묵요리 묵양념장레시피,도토리묵무침,67103259,심플민,15,0,1,무침,일상,기타,메인반찬,간장에 콕 찍어 그냥 먹어도 맛있는 묵을 매콤새콤달콤한 양념장에 각종 야채 더해 무...,[재료] 도토리묵700g| 상추10장(또는 쌈채소)| 당근1/2개|...,4인분,초급,NaN,20241226233432,https://recipe1.ezmember.co.kr/cache/recipe/20...
23188,7041367,묵밥 만드는 법 온묵밥 만들기 도토리묵사발 육수 레시피 도토리묵 요리,묵밥,74288368,포남댁,8,0,0,끓이기,일상,기타,밥/죽/떡,가볍게 먹기 좋으면서 든든한 온묵밥을 만들어 먹었어요. 만들어서 남은 묵을 냉장고에...,[주재료] 도토리묵350g| 국물멸치1줌| 다시마1개| 물500...,3인분,아무나,NaN,20241226233552,https://recipe1.ezmember.co.kr/cache/recipe/20...
23189,7041368,세발나물무침 레시피 세발나물요리,세발나물무침,67103259,심플민,10,0,0,무침,일상,채소류,밑반찬,향기 가득 영양 가득 품은 봄나물 무침으로 대표적인 나물이지만 요즘은 계절에 관계없...,[재료] 세발나물200g| 소금0.5큰술(데칠용)| 국간장0.5큰술...,4인분,초급,30분이내,20241226234147,https://recipe1.ezmember.co.kr/cache/recipe/20...
23190,7041369,제육볶음 맛있게 만드는 법 대패삼겹살로 제육볶음 만들기 고추장 제육볶음 양념 레시피,제육볶음,74288368,포남댁,11,0,0,볶음,일상,돼지고기,메인반찬,대패삼겹살은 얇게 썰어져 있어서 금방 구워 먹기도 편하고 양념을 해서 제육볶음으로 ...,[주재료] 대패삼겹살700g| 느타리버섯300g| 양파1/2개| 청...,4인분,아무나,30분이내,20241226234857,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [14]:
low_ls = ['갈치', '피마늘', '포도', '양파', '새송이버섯', '브로콜리', '배', '딸기', '사과', '단감', '고구마', '굴', '레몬', '망고', '감자', '느타리버섯', '당근', '감귤', '토마토', '체리']

In [30]:
len(low_ls)

20

In [37]:
temp = ['닭가슴살', '연어', '고구마']

In [38]:
df[df['ckg_mtrl_cn'].str.contains('|'.join(temp), na=False)]

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url,low_ls,low_ls_cnt
69,7016908,에어프라이어 고구마스틱 도전기,고구마스틱,dal031,잔근육,418,0,1,굽기,간식,곡류,과자,부드럽고 달콤 담백 바삭한 고구마스틱은 건강한 간식 으로 좋아요.바삭바삭하니 집어 ...,[재료] 고구마2개| 오일2T| 파슬리가루톡톡,3인분,초급,60분이내,20240102143012,https://recipe1.ezmember.co.kr/cache/recipe/20...,[고구마],1
72,7016911,고구마솥밥,고구마솥밥,18528617,다시봄,252,0,0,기타,일상,채소류,밥/죽/떡,고구마가 제철이라서 너무 좋은 요즘 고구마를 듬뿍 넣어서 고구마솥밥을 만들었습니다.,[재료] 고구마3개| 쌀 불린거4인분,4인분,초급,30분이내,20240102144501,https://recipe1.ezmember.co.kr/cache/recipe/20...,[고구마],1
74,7016913,고구마튀김,고구마튀김,18528617,다시봄,471,0,2,튀김,간식,채소류,디저트,겨울간식에 너무나도 만들기도 간단하면서 우리아이들이 좋아하는 간식이라서 자주 만들어...,[재료] 고구마2개| 튀김가루1컵| 식용유넉넉히| 얼음넉넉히,4인분,초급,30분이내,20240102144542,https://recipe1.ezmember.co.kr/cache/recipe/20...,[고구마],1
75,7016914,고구마닭볶음탕,고구마닭볶음탕,18528617,다시봄,963,0,17,조림,일상,닭고기,국/탕,고구마가 제철이라서 좋은 겨울이라서 감자대신 고구마를 넉넉히 넣어서 국물을 자작자작...,[재료] 닭 볶음용1팩| 고구마5개| 대파넉넉히| 양파1개| ...,4인분,초급,30분이내,20240102144646,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 고구마]",2
87,7016926,에어프라이어로 간단하게 백종원 고구마피자 만들기,고구마피자,33787439,눈크미,4004,0,102,굽기,간식,곡류,퓨전,집에 남아있는 고구마로 백종원 고구마피자를 만들어 봤어요~ 에어프라이어를 이용해 쉽...,[재료] 고구마 중1개| 피망1/4개| 양파1/4개| 베이컨4장...,2인분,아무나,30분이내,20240102173906,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 고구마]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23052,7041225,닭곰탕 만들기 닭곰탕 레시피 닭곰탕 만드는법 닭곰탕 다대기 닭곰탕 닭가슴살요리,닭곰탕,daram0913,김씨네쉬운레시피,59,0,0,끓이기,다이어트,닭고기,국/탕,닭곰탕 만들기 닭곰탕 레시피 닭곰탕 만드는법 닭곰탕 다대기 닭곰탕 닭가슴살요리,[재료] 닭가슴살4덩이| 대파1대| 양파1개| 통마늘12알| ...,NaN,아무나,NaN,20241224110559,https://recipe1.ezmember.co.kr/cache/recipe/20...,[양파],1
23083,7041258,든든하고 건강한 누룽지 닭죽,누룽지닭죽,laurenrr,뿡씨스터즈,31,0,10,끓이기,일상,닭고기,밥/죽/떡,든든하고 건강한 한끼 다이어트 메뉴로도 원기회복에 좋은 누룽지 닭죽 만들어 볼게요 ​,[누룽지 닭죽 재료] 누룽지140g| 물1L| 당근1개| 부추1/...,4인분,아무나,30분이내,20241224185923,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 당근]",2
23102,7041277,[연어카르파치오] 훈제 연어로 만든 핑거 푸드,연어카르파치오,93451839,싱글의한끼,25,0,0,기타,손님접대,기타,양식,카르파초는 익히지 않은 쇠고기나 생선을 이용한 이탈리아 요리의 하나인데 카르파치오라...,[재료] 바게트1/2개| 훈제연어200g| 케이퍼 [소스 재료] 그...,4인분,초급,30분이내,20241225090605,https://recipe1.ezmember.co.kr/cache/recipe/20...,[레몬],1
23112,7041289,고구마 요리 고구마전 만들기 아이들 간식 레시피,고구마전,aeaelove,춤추는루나,27,0,0,부침,간식,곡류,기타,고구마 튀김이 부담스러울 때 고구마전 만들기 겨울방학 아이들 간식 레시피 고구마 요...,[재료] 고구마4개(중 사이즈)| 부침가루3| 식용유넉넉히 [반죽 ...,4인분,초급,60분이내,20241225161651,https://recipe1.ezmember.co.kr/cache/recipe/20...,[고구마],1


In [18]:
df[df['ckg_mtrl_cn'].str.contains('|'.join(low_ls), na=False)]

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...
3,7016816,만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품,만두전골,aeaelove,춤추는루나,6350,0,6,끓이기,손님접대,가공식품류,국/탕,속이 꽉 찬 만두가 듬뿍 푸짐한 백종원 만두전골 만두전골 레시피 만두 요리는 다양한...,[재료] 만두12개| 청경채3개| 양파1/2개| 대파1/2개|...,4인분,초급,60분이내,20240101073609,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,7016817,새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기,무수분보쌈,aeaelove,춤추는루나,1829,0,6,삶기,손님접대,돼지고기,메인반찬,촉촉하고 부드러운 통삼겹살 무수분 보쌈 삶는법 무수분 보쌈 삶는 시간 백종원 보쌈 ...,[재료] 통삼겹살600g| 양파1개| 못난이 사과 小2개| 대파2...,4인분,초급,90분이내,20240101074507,https://recipe1.ezmember.co.kr/cache/recipe/20...
6,7016819,돼지 등갈비 바베큐 폭립 만드는법 돼지 등갈비구이,돼지등갈비구이,77721145,써지니,3421,0,8,굽기,일상,돼지고기,메인반찬,등갈비요리는 주로 매운찜으로 요리하여 먹다가 이번에는 색다르게 바베큐 폭립으로 만들...,[재료] 돼지 등갈비550g| 올리브오일2큰술| 통깨살짝 [등갈비삶...,2인분,아무나,60분이내,20240101084230,https://recipe1.ezmember.co.kr/cache/recipe/20...
7,7016820,참치 카나페 만들기 카나페 핑거푸드 만드는 법 참치마요 카나페,참치카나페,77721145,써지니,4259,0,10,기타,일상,기타,디저트,간단하게 술안주 와인안주로 먹기 좋은 카나페 참치마요 카나페를 만들어봤어요.(큰술:...,[재료] 참치 1캔85g| 마요네즈1큰술| 흰설탕2~3꼬집| 후추...,2인분,아무나,10분이내,20240101084413,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181,7041360,간단 아침반찬 양배추요리 /양배추반찬/ 새우양배추볶음/ 양배추새우볶음,새우양배추볶음,loveshine707,쿡따라,27,0,2,볶음,일상,해물류,밑반찬,아이들 밥반찬으로 좋은 양배추반찬 새우양배추볶음입니다 매운 양념 넣지 않고 간장베이...,[재료] 양배추1/5개| 칵테일새우12마리| 당근15g| 식용유2...,3인분,초급,10분이내,20241226223001,https://recipe1.ezmember.co.kr/cache/recipe/20...
23186,7041365,아삭 쫀득한 연근조림 만드는 법 연근 반찬 연근조림 양념 레시피,연근조림,67103259,심플민,22,0,0,조림,일상,채소류,밑반찬,연꽃의 땅속줄기인 연근은 추운 지금부터 봄까지 제철이라 지금 싱싱한 제철 채소로 영...,[재료] 연근500g| 식용유3큰술| 식초1큰술| 물100ml...,6인분이상,초급,60분이내,20241226232723,https://recipe1.ezmember.co.kr/cache/recipe/20...
23187,7041366,도토리묵무침 도토리묵 양념장 만들기 도토리묵요리 묵양념장레시피,도토리묵무침,67103259,심플민,15,0,1,무침,일상,기타,메인반찬,간장에 콕 찍어 그냥 먹어도 맛있는 묵을 매콤새콤달콤한 양념장에 각종 야채 더해 무...,[재료] 도토리묵700g| 상추10장(또는 쌈채소)| 당근1/2개|...,4인분,초급,NaN,20241226233432,https://recipe1.ezmember.co.kr/cache/recipe/20...
23190,7041369,제육볶음 맛있게 만드는 법 대패삼겹살로 제육볶음 만들기 고추장 제육볶음 양념 레시피,제육볶음,74288368,포남댁,11,0,0,볶음,일상,돼지고기,메인반찬,대패삼겹살은 얇게 썰어져 있어서 금방 구워 먹기도 편하고 양념을 해서 제육볶음으로 ...,[주재료] 대패삼겹살700g| 느타리버섯300g| 양파1/2개| 청...,4인분,아무나,30분이내,20241226234857,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [19]:
# low_ls에 포함된 재료들을 찾아서 새로운 컬럼에 추가
df['low_ls'] = df['ckg_mtrl_cn'].apply(lambda x: [item for item in low_ls if item in str(x)])
df[df['low_ls'].str.len() > 0]

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url,low_ls
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 배, 감자]"
3,7016816,만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품,만두전골,aeaelove,춤추는루나,6350,0,6,끓이기,손님접대,가공식품류,국/탕,속이 꽉 찬 만두가 듬뿍 푸짐한 백종원 만두전골 만두전골 레시피 만두 요리는 다양한...,[재료] 만두12개| 청경채3개| 양파1/2개| 대파1/2개|...,4인분,초급,60분이내,20240101073609,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 느타리버섯]"
4,7016817,새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기,무수분보쌈,aeaelove,춤추는루나,1829,0,6,삶기,손님접대,돼지고기,메인반찬,촉촉하고 부드러운 통삼겹살 무수분 보쌈 삶는법 무수분 보쌈 삶는 시간 백종원 보쌈 ...,[재료] 통삼겹살600g| 양파1개| 못난이 사과 小2개| 대파2...,4인분,초급,90분이내,20240101074507,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 사과, 레몬]"
6,7016819,돼지 등갈비 바베큐 폭립 만드는법 돼지 등갈비구이,돼지등갈비구이,77721145,써지니,3421,0,8,굽기,일상,돼지고기,메인반찬,등갈비요리는 주로 매운찜으로 요리하여 먹다가 이번에는 색다르게 바베큐 폭립으로 만들...,[재료] 돼지 등갈비550g| 올리브오일2큰술| 통깨살짝 [등갈비삶...,2인분,아무나,60분이내,20240101084230,https://recipe1.ezmember.co.kr/cache/recipe/20...,[굴]
7,7016820,참치 카나페 만들기 카나페 핑거푸드 만드는 법 참치마요 카나페,참치카나페,77721145,써지니,4259,0,10,기타,일상,기타,디저트,간단하게 술안주 와인안주로 먹기 좋은 카나페 참치마요 카나페를 만들어봤어요.(큰술:...,[재료] 참치 1캔85g| 마요네즈1큰술| 흰설탕2~3꼬집| 후추...,2인분,아무나,10분이내,20240101084413,https://recipe1.ezmember.co.kr/cache/recipe/20...,[토마토]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181,7041360,간단 아침반찬 양배추요리 /양배추반찬/ 새우양배추볶음/ 양배추새우볶음,새우양배추볶음,loveshine707,쿡따라,27,0,2,볶음,일상,해물류,밑반찬,아이들 밥반찬으로 좋은 양배추반찬 새우양배추볶음입니다 매운 양념 넣지 않고 간장베이...,[재료] 양배추1/5개| 칵테일새우12마리| 당근15g| 식용유2...,3인분,초급,10분이내,20241226223001,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[배, 굴, 당근]"
23186,7041365,아삭 쫀득한 연근조림 만드는 법 연근 반찬 연근조림 양념 레시피,연근조림,67103259,심플민,22,0,0,조림,일상,채소류,밑반찬,연꽃의 땅속줄기인 연근은 추운 지금부터 봄까지 제철이라 지금 싱싱한 제철 채소로 영...,[재료] 연근500g| 식용유3큰술| 식초1큰술| 물100ml...,6인분이상,초급,60분이내,20241226232723,https://recipe1.ezmember.co.kr/cache/recipe/20...,[굴]
23187,7041366,도토리묵무침 도토리묵 양념장 만들기 도토리묵요리 묵양념장레시피,도토리묵무침,67103259,심플민,15,0,1,무침,일상,기타,메인반찬,간장에 콕 찍어 그냥 먹어도 맛있는 묵을 매콤새콤달콤한 양념장에 각종 야채 더해 무...,[재료] 도토리묵700g| 상추10장(또는 쌈채소)| 당근1/2개|...,4인분,초급,NaN,20241226233432,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 당근]"
23190,7041369,제육볶음 맛있게 만드는 법 대패삼겹살로 제육볶음 만들기 고추장 제육볶음 양념 레시피,제육볶음,74288368,포남댁,11,0,0,볶음,일상,돼지고기,메인반찬,대패삼겹살은 얇게 썰어져 있어서 금방 구워 먹기도 편하고 양념을 해서 제육볶음으로 ...,[주재료] 대패삼겹살700g| 느타리버섯300g| 양파1/2개| 청...,4인분,아무나,30분이내,20241226234857,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 느타리버섯]"


In [22]:
df.loc[:, 'low_ls_cnt'] = df['low_ls'].str.len()

In [24]:
con = df['low_ls_cnt'] > 0
temp = df[con]

In [29]:
temp

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url,low_ls,low_ls_cnt
2,7016815,우거지감자탕 뼈해장국 끓이는법,우거지감자탕,87771622,김한솔,4008,0,29,끓이기,해장,돼지고기,국/탕,까다로운 남편의 입맛을 맞추기위해 여러번 시도끝에 만들어낸 최적의 레시피입니다. 한...,[재료] 돼지등뼈1.5kg| 양파1/2개| 감자1개| 대파1대...,4인분,중급,2시간이내,20240101020501,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 배, 감자]",3
3,7016816,만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품,만두전골,aeaelove,춤추는루나,6350,0,6,끓이기,손님접대,가공식품류,국/탕,속이 꽉 찬 만두가 듬뿍 푸짐한 백종원 만두전골 만두전골 레시피 만두 요리는 다양한...,[재료] 만두12개| 청경채3개| 양파1/2개| 대파1/2개|...,4인분,초급,60분이내,20240101073609,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 느타리버섯]",2
4,7016817,새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기,무수분보쌈,aeaelove,춤추는루나,1829,0,6,삶기,손님접대,돼지고기,메인반찬,촉촉하고 부드러운 통삼겹살 무수분 보쌈 삶는법 무수분 보쌈 삶는 시간 백종원 보쌈 ...,[재료] 통삼겹살600g| 양파1개| 못난이 사과 小2개| 대파2...,4인분,초급,90분이내,20240101074507,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 사과, 레몬]",3
6,7016819,돼지 등갈비 바베큐 폭립 만드는법 돼지 등갈비구이,돼지등갈비구이,77721145,써지니,3421,0,8,굽기,일상,돼지고기,메인반찬,등갈비요리는 주로 매운찜으로 요리하여 먹다가 이번에는 색다르게 바베큐 폭립으로 만들...,[재료] 돼지 등갈비550g| 올리브오일2큰술| 통깨살짝 [등갈비삶...,2인분,아무나,60분이내,20240101084230,https://recipe1.ezmember.co.kr/cache/recipe/20...,[굴],1
7,7016820,참치 카나페 만들기 카나페 핑거푸드 만드는 법 참치마요 카나페,참치카나페,77721145,써지니,4259,0,10,기타,일상,기타,디저트,간단하게 술안주 와인안주로 먹기 좋은 카나페 참치마요 카나페를 만들어봤어요.(큰술:...,[재료] 참치 1캔85g| 마요네즈1큰술| 흰설탕2~3꼬집| 후추...,2인분,아무나,10분이내,20240101084413,https://recipe1.ezmember.co.kr/cache/recipe/20...,[토마토],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181,7041360,간단 아침반찬 양배추요리 /양배추반찬/ 새우양배추볶음/ 양배추새우볶음,새우양배추볶음,loveshine707,쿡따라,27,0,2,볶음,일상,해물류,밑반찬,아이들 밥반찬으로 좋은 양배추반찬 새우양배추볶음입니다 매운 양념 넣지 않고 간장베이...,[재료] 양배추1/5개| 칵테일새우12마리| 당근15g| 식용유2...,3인분,초급,10분이내,20241226223001,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[배, 굴, 당근]",3
23186,7041365,아삭 쫀득한 연근조림 만드는 법 연근 반찬 연근조림 양념 레시피,연근조림,67103259,심플민,22,0,0,조림,일상,채소류,밑반찬,연꽃의 땅속줄기인 연근은 추운 지금부터 봄까지 제철이라 지금 싱싱한 제철 채소로 영...,[재료] 연근500g| 식용유3큰술| 식초1큰술| 물100ml...,6인분이상,초급,60분이내,20241226232723,https://recipe1.ezmember.co.kr/cache/recipe/20...,[굴],1
23187,7041366,도토리묵무침 도토리묵 양념장 만들기 도토리묵요리 묵양념장레시피,도토리묵무침,67103259,심플민,15,0,1,무침,일상,기타,메인반찬,간장에 콕 찍어 그냥 먹어도 맛있는 묵을 매콤새콤달콤한 양념장에 각종 야채 더해 무...,[재료] 도토리묵700g| 상추10장(또는 쌈채소)| 당근1/2개|...,4인분,초급,NaN,20241226233432,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 당근]",2
23190,7041369,제육볶음 맛있게 만드는 법 대패삼겹살로 제육볶음 만들기 고추장 제육볶음 양념 레시피,제육볶음,74288368,포남댁,11,0,0,볶음,일상,돼지고기,메인반찬,대패삼겹살은 얇게 썰어져 있어서 금방 구워 먹기도 편하고 양념을 해서 제육볶음으로 ...,[주재료] 대패삼겹살700g| 느타리버섯300g| 양파1/2개| 청...,4인분,아무나,30분이내,20241226234857,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 느타리버섯]",2


In [28]:
temp['ckg_knd_acto_nm'].value_counts()

ckg_knd_acto_nm
메인반찬        2900
밑반찬         2170
밥/죽/떡       1413
국/탕          976
기타           734
면/만두         711
샐러드          631
찌개           610
김치/젓갈/장류     434
빵            427
양식           408
디저트          403
퓨전           288
차/음료/술       216
양념/소스/잼      182
스프           113
과자            44
Name: count, dtype: int64

In [33]:
temp[temp['rcp_ttl'].str.contains(r'건강|간편|다이어트')]

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url,low_ls,low_ls_cnt
34,7016866,알배추 두부 쌈 으로 새해아침 특별하고 건강하게,알배추두부쌈,cheonsa12,요리재능발견,757,0,9,부침,일상,채소류,메인반찬,두부와 배추는 건강하게 만드는 재료이죠 두 가지를 이용해서 쌈 만들었어요 쌈 속에 ...,[재료] 알배추15장| 두부230g| 소고기80g| 숙주120g...,3인분,아무나,30분이내,20240101205508,https://recipe1.ezmember.co.kr/cache/recipe/20...,[배],1
57,7016891,새해에 먹는 건강 배추전,배추전,32865661,우수푸드,315,0,15,부침,술안주,채소류,디저트,마른새우를 갈아서 넣은 배추전은 고소하고 맛있어요~ 건새우가 없다면 배추만해서 드셔...,[재료] 배추6장| 부침가루3.5컵| 소금적당량| 식용유적당량...,3인분,초급,60분이내,20240102120947,https://recipe1.ezmember.co.kr/cache/recipe/20...,[배],1
93,7016937,다이어트 곤약크림파스타 | 키토파스타,곤약크림파스타,17917086,온스맘의키토레시피,501,0,16,볶음,다이어트,달걀/유제품,양식,곤약면으로 요리할때 음식이 따로 놀았나요? 곤약면으로 꾸덕하고 진한 크림파스타 만들...,[재료] 파스타면200g(곤약면)| 올리브오일조금| 양파85g| 새...,1인분,아무나,15분이내,20240102204551,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 새송이버섯]",2
104,7016948,편스토랑 진서연 양송이햄버거 핑거푸드 홈파티메뉴 와인안주 손님초대요리 다이어트 메뉴...,양송이햄버거,ainee77,윤채맘,1230,0,32,기타,손님접대,채소류,디저트,다이어트 중에 먹어도 부담없고 와인안주로 먹어도 좋은 담백한 핑거푸드 양송이버거 만...,[재료] 양송이버섯12개| 냉동새우5마리| 오이1/2개| 방울토마토...,2인분,아무나,15분이내,20240103003759,https://recipe1.ezmember.co.kr/cache/recipe/20...,[토마토],1
118,7016962,다이어트 메뉴 시금치또띠아피자 만들기 시금치프리타타 레시피,시금치프리타타,63887139,효땡,878,0,13,굽기,간식,채소류,디저트,겉바속촉 너무 맛있는 시금치 또띠아피자 레시피 소개해요~,[재료] 또띠아1장| 시금치| 방울토마토2-3개| 체다치즈1장...,2인분,아무나,30분이내,20240103092059,https://recipe1.ezmember.co.kr/cache/recipe/20...,[토마토],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22871,7041032,다이어트 돼지목살 양배추 파채 무침/ 다이어트 요리 레시피,돼지목살양배추파채무침,0807twins,배고픈엄마,33,0,0,기타,일상,돼지고기,메인반찬,제육볶음용 돼지고기 목살이 냉동실에 있어 해동한 후 끓는 물에 넣고 삶아 샤브샤브용...,[재료] 제육용 목살250g| 통후추1작은술| 월계수3장| 채썬 양...,NaN,아무나,NaN,20241221121120,https://recipe1.ezmember.co.kr/cache/recipe/20...,[배],1
22886,7041048,초 간단 다이어트 순두부 치즈 그라탕 만들기,순두부치즈그라탕,28603415,다다쏭,37,0,0,찜,초스피드,달걀/유제품,퓨전,너무 힘들어서 요리고 뭐고 배달 시키고 싶은 날 진짜 간단하게 먹는 한끼 메뉴에요 ...,[재료] 양배추200g(2 줌)| 순두부1/2봉지| 계란1개| 다진...,1인분,아무나,15분이내,20241221181939,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[배, 토마토]",2
23083,7041258,든든하고 건강한 누룽지 닭죽,누룽지닭죽,laurenrr,뿡씨스터즈,31,0,10,끓이기,일상,닭고기,밥/죽/떡,든든하고 건강한 한끼 다이어트 메뉴로도 원기회복에 좋은 누룽지 닭죽 만들어 볼게요 ​,[누룽지 닭죽 재료] 누룽지140g| 물1L| 당근1개| 부추1/...,4인분,아무나,30분이내,20241224185923,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 당근]",2
23168,7041347,다이어트 레시피로 좋은 간단한 계란 당근 김밥 만들기,계란당근김밥,55836534,쿠킹연수,20,0,0,부침,다이어트,채소류,밥/죽/떡,시장에서 맛있는 키토 김밥을 몇 번 사 먹다가... 날이 추워진 만큼 가기도 힘들고...,[재료] 당근1개| 계란3개| 단무지2줄| 김1.5장| 다진마...,1인분,중급,30분이내,20241226172233,https://recipe1.ezmember.co.kr/cache/recipe/20...,[당근],1


In [36]:
temp[temp['ckg_knd_acto_nm'].isin(['메인반찬', '밑반찬'])].sort_values(by='inq_cnt', ascending=False)

,rcp_sno,rcp_ttl,ckg_nm,rgtr_id,rgtr_nm,inq_cnt,rcmm_cnt,srap_cnt,ckg_mth_acto_nm,ckg_sta_acto_nm,ckg_mtrl_acto_nm,ckg_knd_acto_nm,ckg_ipdc,ckg_mtrl_cn,ckg_inbun_nm,ckg_dodf_nm,ckg_time_nm,first_reg_dt,rcp_img_url,low_ls,low_ls_cnt
5398,7022686,국물 닭볶음탕 황금레시피,국물닭볶음탕,81748884,요리하는쮸네,45495,0,35,끓이기,일상,닭고기,메인반찬,국물 닭볶음탕 황금레시피 알려드릴게요 : ),[재료] 절단 닭1kg| 대파1대| 양파1/2개| 당근1/2개...,2인분,아무나,60분이내,20240321110944,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 감자, 당근]",3
4346,7021550,편스토랑 류수영 고등어조림 레시피 어남선생 고등어무조림 생선조림 양념장,고등어무조림,77721145,써지니,34821,0,230,조림,일상,기타,메인반찬,고등어조림 생선조림을 종종 만들어 먹는데 방송에서 본 편스토랑 류수영 님 고등어무조...,[재료] 고등어2마리(450g)| 무500g| 양파1개(190g)|...,2인분,아무나,60분이내,20240307053735,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 굴]",2
3222,7020351,돼지고기 수육 맛있게 삶는법 수육만들기 돼지고기 앞다리살 수육 삶는시간 레시피,돼지고기수육,17942493,귀차니,31764,0,12,찜,일상,돼지고기,메인반찬,돼지고기 수육 맛있게 삶는법 수육만들기 돼지고기 앞다리살 수육 삶는시간 레시피,[재료] 돼지고기 앞다리살1.2kg| 된장2스푼| 미림2스푼| 월계...,NaN,아무나,NaN,20240219211953,https://recipe1.ezmember.co.kr/cache/recipe/20...,[양파],1
9382,7026944,안부서지는 감자조림 맛있게 만드는법 /감자조림 레시피 /감자조림 만드는 법 /감자조...,감자조림,14524851,두율이아빠,28093,0,39,조림,일상,기타,밑반찬,안부서지는 감자조림 맛있게 만드는법 /감자조림 레시피 /감자조림 만드는 법 /감자조...,[재료] 감자4개(감자 작은크기 300~320g)| 꽈리고추12개(50~...,NaN,초급,NaN,20240523130839,https://recipe1.ezmember.co.kr/cache/recipe/20...,[감자],1
7864,7025309,알토란 오이소박이 만드는법 통오이소박이 담그는법,오이소박이,lovingyou002,집밥먹는깡주부,26420,0,11,무침,일상,채소류,밑반찬,오랜만에 아삭아삭한 오이소박이 생각이나서 후다닥 만들어봤어요 날이 조금씩 더워지니 ...,[재료] 오이4~5개 [속재료] 부추1줌| 당근1/4개| 양파1/...,4인분,초급,30분이내,20240427170816,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 당근]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23191,7041370,애호박 계란말이 만드는 법 달걀말이 계란 요리 레시피,애호박계란말이,33866137,동그랑,11,0,0,부침,일상,달걀/유제품,밑반찬,애호박 넣고 계란말이 만들었어요,[재료] 계란7개| 애호박1/4개| 당근1/5개| 쪽파3대(또...,2인분,초급,NaN,20241226235311,https://recipe1.ezmember.co.kr/cache/recipe/20...,[당근],1
22782,7040938,동남아 여행 하는 느낌! “커민 두부 볶음”,커민두부볶음,25766193,유지영♡♡,11,0,0,볶음,술안주,콩/견과류,메인반찬,커민(큐민)이라는 가루를 이용하는 요리를 찾던 중 알게 된 요리예요! 건강에도 좋고...,[재료] 두부1모| 양파1개(슬라이스)| 브로콜리2-3개(잘게썬것)...,2인분,초급,10분이내,20241219212051,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[양파, 브로콜리]",2
23141,7041318,부추양파무침 레시피,부추양파무침,17672780,라피네,11,0,0,무침,일상,채소류,밑반찬,부추양파무침 레시피,[재료] 부추1줌| 양파1/2개 [양념] 고추가루2T| 진간장3...,2인분,초급,10분이내,20241226085549,https://recipe1.ezmember.co.kr/cache/recipe/20...,[양파],1
23086,7041261,라조장을 활용한 매콤한 돼지고기산적 만들기,돼지고기산적,jsblack,나봄수니,11,0,0,조림,기타,돼지고기,메인반찬,제사상에 올라가는 고기산적은 간장베이스로 올리잖아요. 그래서 한번 매콤하게 만들어 ...,[재료] 돼지고기 산적1개(260g)| 통깨약간 [양념] 다진대파1큰...,1인분,아무나,NaN,20241224213743,https://recipe1.ezmember.co.kr/cache/recipe/20...,[굴],1
